In [1]:
import os
import json
import tqdm
import torch
import datetime
import argparse
import pycocotools.mask as cocomask
from detectron2.utils.file_io import PathManager

In [2]:
import numpy as np
from PIL import Image
import matplotlib.patches as mpatches

In [3]:
INFO = {
    "description": "ImageNet-1K: Self-train",
    "url": "",
    "version": "1.0",
    "year": 2022,
    "contributor": "Xudong Wang",
    "date_created": datetime.datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "Apache License",
        "url": "https://github.com/facebookresearch/CutLER/blob/main/LICENSE"
    }
]

CATEGORIES = [
    {
        'id': 1,
        'name': 'fg',
        'supercategory': 'fg',
    },
]

new_dict_filtered = {
    "info": INFO,
    "licenses": LICENSES,
    "categories": CATEGORIES,
    "images": [],
    "annotations": []
}

category_info = {
    "is_crowd": 0,
    "id": 1
}

In [4]:
def segmToRLE(segm, h, w):
    if isinstance(segm, list):
        # polygon -- a single object might consist of multiple parts
        # we merge all parts into one mask rle code
        rles = cocomask.frPyObjects(segm, h, w)
        rle = cocomask.merge(rles)
    elif isinstance(segm["counts"], list):
        # uncompressed RLE
        rle = cocomask.frPyObjects(segm, h, w)
    else:
        # rle
        rle = segm
    return rle

def rle2mask(rle, height, width):
    if "counts" in rle and isinstance(rle["counts"], list):
        # if compact RLE, ignore this conversion
        # Magic RLE format handling painfully discovered by looking at the
        # COCO API showAnns function.
        rle = cocomask.frPyObjects(rle, height, width)
    mask = cocomask.decode(rle)
    return mask

def cocosegm2mask(segm, h, w):
    rle = segmToRLE(segm, h, w)
    mask = rle2mask(rle, h, w)
    return mask

In [5]:
# if __name__ == "__main__":
# load model arguments
    # parser = argparse.ArgumentParser(description='Generate labelmaps from json files')
    # parser.add_argument('--ann', type=str, 
    #                     default='DETECTRON2_DATASETS/carotid-mini/annotations/imagenet_train_fixsize480_tau0.15_N3.json',
    #                     help='Path to maskcut annotation or model predictions')
    # parser.add_argument('--dataset', type=str,    
    #                       default='DETECTRON2_DATASETS/carotid-mini/images',
    #                     help='Path to the dataset')
    # parser.add_argument('--save_path', type=str,
    #                     default='DETECTRON2_DATASETS/carotid-mini/labelmaps',
    #                     help='Path to save the generated labelmaps')
    # parser.add_argument('--threshold', type=float, default=0.5,
    #                     help='Confidence score thresholds')
    # args = parser.parse_args()

In [6]:
DETECTRON2_DATASETS="/home/guests/oleksandra_tmenova/test/project/thesis-codebase/data"


In [7]:
# ann_path = DETECTRON2_DATASETS+'/carotid-mini/annotations/imagenet_train_fixsize480_tau0.15_N3.json'
ann_path ="/home/guests/oleksandra_tmenova/test/project/thesis-codebase/data/carotid-mini/annotations/imagenet_train_fixsize480_tau0.15_N3.json"

In [8]:
save_path = DETECTRON2_DATASETS + "/carotid-mini/labelmaps"

In [9]:
save_path

'/home/guests/oleksandra_tmenova/test/project/thesis-codebase/data/carotid-mini/labelmaps'

In [10]:
dataset_path = DETECTRON2_DATASETS + '/carotid-mini/images'

In [11]:
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [12]:
# load annotations
# ann_dict = json.load(open(ann_path))
with PathManager.open(ann_path, "r") as f:
    ann_dict = json.load(f)
image_list = ann_dict['images']
annotations = ann_dict['annotations']

In [13]:
image_list[0]

{'id': 1,
 'file_name': 'train/img0001.jpg',
 'width': 256,
 'height': 256,
 'date_captured': '2023-08-29 11:47:27.705046',
 'license': 1,
 'coco_url': '',
 'flickr_url': ''}

In [14]:
# group annotations by images in a new dictionary
image_to_anns = {}
for id, ann in enumerate(annotations):
    if ann['image_id'] in image_to_anns:
        image_to_anns[ann['image_id']].append(ann)
    else:
        image_to_anns[ann['image_id']] = [ann]

In [15]:
image_to_anns[1]

[{'id': 1,
  'image_id': 1,
  'category_id': 1,
  'iscrowd': 0,
  'area': 13939,
  'bbox': [7.0, 38.0, 186.0, 151.0],
  'segmentation': {'size': [256, 256],
   'counts': 'ok11b02e62UI4g6HnH;5Nj6GQI?2Kl6h0N1O10N1001OL211003FnN`IR1`6QO]IP1a6QO_Io0b66iNfNjK\\1T4dNlK\\1T4eNkK\\1T4dNkK]1U4cNiK`1V4aN[K2Aa1T5]NYK4B`1U5\\NXK6@a1X5YNVKU2g4kMWKX2h4iMVKX2j4hMUKZ2j4fMUK\\2IaM<1R42iKa2H\\M91HO[43gKNNh2OWM24OL[43fKU3OhL[4^3fKbLZ4^3fKcLV4`3jKbLS4_3mKaLS4_3mK`LT4_3701O0O1000O101O1O000O0100O10O100001O000O1O2N1O1O10O01K500N2O1O100N20O100000O1000000000000O1000000000O100000N2N1100O2N10000O1NUOUKkM0g0j4^1[KaNe4^1]KaNb4_1_K`Na4a1_K_N^4d1aK]N_4c1aK\\N_4d1bKXNb4g1_KYNa4f1`K[N_4e1aK\\N^4c1cKaNY4^1iKaNW4_1iKaNW4_1iKaNX4^1iK`NZ4^1hK^NZ4b1fKWN@J0Mj4R2gKVN`4i1aKRN@No4P2aKRNAMn4P2bKSNc4i1jJTNe02b4i1`KXN`4g1]KVN]O3V5h1kJVN<6i4d1kJVN:9j4a1kJXN:8i4a1nJVN9:h4_1QKWN6:i4_1PKXN79i4`1oJWN98g4b1PKUN:6DIR5l1PKTN;6i4f1lJTN;5j4f1lJUN:3CLW5l1mJTN93DMU5l1^KTN\\O0Z5l1YKUNi4j1XKUNh4j1h000O10mIWNn5j1QJWNo5h1RJXNm5i15O1N2O001O1O0001

In [16]:
image_to_labelmaps = {}

In [26]:
def masks2labelmap(masks: list, h: int, w: int):
    # print(len(masks))
    labelmap = np.zeros((h, w), dtype=int) #Check if order h, w is correct
    label = 1 #start from 1, to leave 0 be a background index in the labelmap
    # n_0s = 0
    # n_1s = 0
    for mask in masks:
        # n_0s = n_0s + (mask==0).sum()
        # n_1s = n_1s + (mask==1).sum()
        # print(mask.shape, "0s:", (mask==0).sum(),"1s:", (mask==1).sum())
        labelmap[mask==1] = label
        label = label + 1
    print("Labelmap: 0s:", (labelmap==0).sum(),"1s:", (labelmap==1).sum(), "2s:", (labelmap==2).sum())
    return labelmap

In [27]:
for k, anns in tqdm.tqdm(image_to_anns.items()):
    if k>1:
        break
    # print(k)
    # get a list of binary masks
    masks = []
    for ann in anns:
        segm = ann['segmentation']
        mask = cocosegm2mask(segm, segm['size'][0], segm['size'][1])
        masks.append(mask)

    # since anns are grouped per image, can get h,w, name from any ann in the list
    h = anns[0]['height']
    w = anns[0]['width']
    # TODO: figure out a better way to get the actual filename of th eimage (stored in image_list)
    # E.g. can build the original im_ann dictionary using image names, not ids
    image_id = anns[0]['image_id'] 
    

    # generate labelmaps
    labelmap = masks2labelmap(masks, h, w)
    image_to_labelmaps[k] = labelmap

    # save labelmap
    output_file = save_path + f'/{image_id}.png'
    # Image.fromarray(labelmap).save(output_file)
    # labelmap_im = Image.fromarray(labelmap.astype(np.uint8)).convert('L')
    labelmap_im = Image.fromarray(np.uint8(labelmap*255)).convert('L')
    # print(labelmap_im)
    labelmap_im.save(output_file)
    
    

    

  6%|▌         | 1/18 [00:00<00:00, 246.69it/s]

(256, 256) 0s: 51597 1s: 13939
(256, 256) 0s: 57305 1s: 8231
Labelmap: 0s: 43366 1s: 13939 2s: 8231


In [23]:
# pred = image_to_labelmaps[1]

# values = np.unique(pred.ravel())
# im = plt.imshow(pred)
# plt.axis(False)
# colors = [ im.cmap(im.norm(value)) for value in values]
# # create a patch (proxy artist) for every color 
# patches = [ mpatches.Patch(color=colors[i], label="{l}".format(l=values[i]) ) for i in range(len(values)) ]
# # put those patched as legend-handles into the legend
# plt.legend(handles=patches, bbox_to_anchor=(1., 1), loc=2)

In [ ]:
# image_to_labelmaps[1]